In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Charger les données
file_path = 'C:/Users/Utilisateur/Desktop/credit-banque/loan-data-673b233f1c1cb921157550.csv'
data = pd.read_csv(file_path)

# Séparer les variables indépendantes (X) et la variable cible (y)
X = data.drop(columns=['Loan_Status'])
y = data['Loan_Status']

# Vérification des types de données
print("Types de données avant encodage :")
print(X.dtypes)

# Encoder les variables catégorielles
label_encoder = LabelEncoder()
X['Education'] = label_encoder.fit_transform(X['Education'])  # Exemple d'encodage
X = pd.get_dummies(X, drop_first=True)  # One-Hot Encoding pour les variables restantes

# Vérification des types de données après encodage
print("\nTypes de données après encodage :")
print(X.dtypes)

# Encodage de la variable cible 'Loan_Status'
y = label_encoder.fit_transform(y)

# Séparation des données en ensembles d'entraînement et de test (80% pour l'entraînement et 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vérification des NaN et des valeurs infinies dans X_train
print("\nVérification des NaN et des valeurs infinies :")
print(X_train.isnull().sum())  # Vérifier les NaN
print(np.isinf(X_train).sum())  # Vérifier les infinies

# Si des NaN ou des infinies sont présentes, les traiter
X_train.fillna(X_train.mean(), inplace=True)  # Remplacer les NaN par la moyenne
X_train = X_train.replace([np.inf, -np.inf], np.nan).dropna()  # Supprimer les valeurs infinies

# Aligner les colonnes de X_train et X_test après l'encodage
X_train, X_test = X_train.align(X_test, join='left', axis=1)

# Ajouter les colonnes manquantes dans X_test avec des zéros
for col in X_train.columns:
    if col not in X_test.columns:
        X_test[col] = 0

# Vérification après l'alignement
print(f"Colonnes de X_train : {X_train.columns}")
print(f"Colonnes de X_test : {X_test.columns}")

# Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Vérification des NaN et des valeurs infinies après normalisation
print(f"NaN dans X_train_scaled : {np.isnan(X_train_scaled).sum()}")
print(f"NaN dans X_test_scaled : {np.isnan(X_test_scaled).sum()}")
print(f"Infinis dans X_train_scaled : {np.isinf(X_train_scaled).sum()}")
print(f"Infinis dans X_test_scaled : {np.isinf(X_test_scaled).sum()}")

# Initialisation des modèles
models = {
    'Logistic Regression': LogisticRegression(max_iter=500, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Entraînement et évaluation des modèles
for model_name, model in models.items():
    # Entraînement
    model.fit(X_train_scaled, y_train)
    
    # Prédictions sur l'ensemble de test
    y_pred = model.predict(X_test_scaled)
    
    # Évaluation des performances
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name} - Accuracy: {accuracy:.4f}")
    print(f"Classification Report for {model_name}:\n{classification_report(y_test, y_pred)}\n")

Types de données avant encodage :
Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
dtype: object

Types de données après encodage :
Education                    int32
ApplicantIncome              int64
CoapplicantIncome          float64
LoanAmount                 float64
Loan_Amount_Term           float64
                            ...   
Dependents_2                  bool
Dependents_3+                 bool
Self_Employed_Yes             bool
Property_Area_Semiurban       bool
Property_Area_Urban           bool
Length: 994, dtype: object

Vérification des NaN et des valeurs infinies :
Education                   0
ApplicantIncome             0
CoapplicantIncome           0
LoanAmount         

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values